<a href="https://colab.research.google.com/github/sarjakpatel/SJSU/blob/main/CMPE%20258/Assignment%202/TensorOperationsUsingTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE 258 Assignment 2 : Tensor Operations using TensorFlow

In [2]:
import tensorflow as tf

**Basic Operations on tf.Tensor object:**

Addition:

In [3]:
print(tf.add(10, 11))
print(tf.add([5, 6], [10, 11]))

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor([15 17], shape=(2,), dtype=int32)


In [4]:
print(tf.square(8))
print(tf.reduce_sum([4, 5, 6]))

tf.Tensor(64, shape=(), dtype=int32)
tf.Tensor(15, shape=(), dtype=int32)


operator overloading:

In [5]:
print(tf.square(5) + tf.square(6)) #

tf.Tensor(61, shape=(), dtype=int32)


Matrix Multiplication:

In [6]:
matrix = tf.matmul([[1]], [[2, 3]])
print(matrix)
print(matrix.shape)
print(matrix.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


**Scalar Tensor:**

In [7]:
scalar_tensor = tf.constant(1)
print(scalar_tensor)

tf.Tensor(1, shape=(), dtype=int32)


**Vector Tensor:**

In [8]:
vector_tensor = tf.constant([1.0, 2.0, 3.0, 4.0])
print(vector_tensor)

tf.Tensor([1. 2. 3. 4.], shape=(4,), dtype=float32)


**Matrix Tensor:**

In [11]:
matrix_tensor1 =  tf.constant([[14, 15],[13, 12]]) # this will create a 2x2 matrix

matrix_tensor2 =  tf.constant([[11, 3], [6,4]])

matrix_tensor3 =  tf.constant([[3, 9], [22,11],[55,66]])

In [12]:
# Largest value inside the matrix tensor
print("Largest value of tensor 1 ",tf.reduce_max(matrix_tensor1))
print("Largest value of tensor 2 ",tf.reduce_max(matrix_tensor2))
# Index of the largest value present inside the matrix tensor
print("index of Largest value of tensor 1",tf.argmax(matrix_tensor1))
print("index of Largest value of tensor 2",tf.argmax(matrix_tensor2))
# Arithmetic Addition of 2 tensors
print("Arithmetic Addition",tf.add(matrix_tensor1, matrix_tensor2), "\n")
# Multiplication of 2 tensors
print("Multiplication",tf.multiply(matrix_tensor1, matrix_tensor2), "\n")
# Matrix Multiplication
print("Matrix Multiplication",tf.matmul(matrix_tensor1, matrix_tensor2), "\n")

Largest value of tensor 1  tf.Tensor(15, shape=(), dtype=int32)
Largest value of tensor 2  tf.Tensor(11, shape=(), dtype=int32)
index of Largest value of tensor 1 tf.Tensor([0 0], shape=(2,), dtype=int64)
index of Largest value of tensor 2 tf.Tensor([0 1], shape=(2,), dtype=int64)
Arithmetic Addition tf.Tensor(
[[25 18]
 [19 16]], shape=(2, 2), dtype=int32) 

Multiplication tf.Tensor(
[[154  45]
 [ 78  48]], shape=(2, 2), dtype=int32) 

Matrix Multiplication tf.Tensor(
[[244 102]
 [215  87]], shape=(2, 2), dtype=int32) 



In [13]:
print("Type of every element:", matrix_tensor1.dtype)
print("Number of axes:", matrix_tensor1.ndim)
print("Shape of tensor:", matrix_tensor1.shape)
print("Elements along axis 0 of tensor:", matrix_tensor1.shape[0])
print("Elements along the last axis of tensor:", matrix_tensor1.shape[-1])
print("Total number of elements (3*2*4*5): ", tf.size(matrix_tensor1).numpy())

Type of every element: <dtype: 'int32'>
Number of axes: 2
Shape of tensor: (2, 2)
Elements along axis 0 of tensor: 2
Elements along the last axis of tensor: 2
Total number of elements (3*2*4*5):  4


**Indexing:**

In [15]:
tensor1 = tf.zeros([1,2,3])

print("Integer present:", tensor1.ndim)

print("Shape :", tensor1.shape)

print("Elements along second axis of tensor:", tensor1.shape[2])

print("Elements along the last axis of tensor:", tensor1.shape[-1])

Integer present: 3
Shape : (1, 2, 3)
Elements along second axis of tensor: 3
Elements along the last axis of tensor: 3


In [16]:
# Slicing of the indices to get a specific portion of the tensor
print("The tensor displayed is :", matrix_tensor3[:].numpy())

print("From 3, before the second last element:", matrix_tensor3[2:7].numpy())
print("Reversed tensor is displayed as :", matrix_tensor3[::-1].numpy())

The tensor displayed is : [[ 3  9]
 [22 11]
 [55 66]]
From 3, before the second last element: [[55 66]]
Reversed tensor is displayed as : [[55 66]
 [22 11]
 [ 3  9]]


**Size Manipulation:**

In [17]:
# Shape returns a `TensorShape` object that shows the size along each axis
x = tf.constant([[1], [2], [3],[4],[5],[6]])
print("The size of a tensor is ",x.shape)

The size of a tensor is  (6, 1)


In [18]:
reshape = tf.reshape(x, [1, 6])
print("The size  of a reshaped tensor x is ",reshape.shape)
print("The reshaped tensor is", reshape)

The size  of a reshaped tensor x is  (1, 6)
The reshaped tensor is tf.Tensor([[1 2 3 4 5 6]], shape=(1, 6), dtype=int32)


**Conversion between NumPy and Tensorflow**

In [ ]:
import numpy as np

In [ ]:
numpyArray = np.ones([2, 2])

In [ ]:
tensor = tf.multiply(numpyArray, 100)
print(tensor)

tf.Tensor(
[[100. 100.]
 [100. 100.]], shape=(2, 2), dtype=float64)


In [ ]:
print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

The .numpy() method explicitly converts a Tensor to a numpy array
[[100. 100.]
 [100. 100.]]


**Broadcasting :**

In [19]:
x = tf.constant([1, 1, 1]) # a unit tensor 

y = tf.constant(2)

# All of these are the same computation
print(tf.multiply(x, 2))
print(x * y)
# both operations will multiply all the elements by 2, here the size will be the same

tf.Tensor([2 2 2], shape=(3,), dtype=int32)
tf.Tensor([2 2 2], shape=(3,), dtype=int32)


**GPU acceleration**

In [ ]:
x = tf.random.uniform([2, 2])

print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0:  
True


In [ ]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(5):
    tf.matmul(x, x)

  result = time.time()-start

  print("5 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("Matrix Multiplication On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([5000, 5000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.list_physical_devices("GPU"):
  print("Matrix Multiplication On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([5000, 5000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

Matrix Multiplication On CPU:
5 loops: 17393.15ms
Matrix Multiplication On GPU:
5 loops: 1.19ms


 tensorflow functions on a dataset:

In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

In [ ]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [ ]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([ 9 25], shape=(2,), dtype=int32)
tf.Tensor([16 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)


**Einsum Operations:**

In [ ]:
m0 = tf.random.normal(shape=[2, 4])
m1 = tf.random.normal(shape=[4, 5])
e = tf.einsum('ij,jk->ik', m0, m1)
# output[i,k] = sum_j m0[i,j] * m1[j, k]
print(e.shape)


(2, 5)


In [ ]:
e = tf.einsum('ij,jk', m0, m1)  # output[i,k] = sum_j m0[i,j] * m1[j, k]
print(e.shape)

(2, 5)


In [ ]:
u = tf.random.normal(shape=[5])
v = tf.random.normal(shape=[5])
e = tf.einsum('i,i->', u, v)  # output = sum_i u[i]*v[i]
print(e.shape)

()


In [ ]:
u = tf.random.normal(shape=[3])
v = tf.random.normal(shape=[5])
e = tf.einsum('i,j->ij', u, v)  # output[i,j] = u[i]*v[j]
print(e.shape)

(3, 5)


In [ ]:
m = tf.ones(2,3)
e = tf.einsum('ij->ji', m0)  # output[j,i] = m0[i,j]
print(e.shape)

(4, 2)


In [ ]:
m = tf.reshape(tf.range(9), [3,3])
diag = tf.einsum('ii->i', m)
print(diag.shape)

(3,)


In [ ]:
trace = tf.einsum('ii', m)  # output[j,i] = trace(m) = sum_i m[i, i]
assert trace == sum(diag)
print(trace.shape)

()


In [ ]:
s = tf.random.normal(shape=[7,5,3])
t = tf.random.normal(shape=[7,3,2])
e = tf.einsum('bij,bjk->bik', s, t)
# output[a,i,k] = sum_j s[a,i,j] * t[a, j, k]
print(e.shape)

(7, 5, 2)


In [ ]:
s = tf.random.normal(shape=[11, 7, 5, 3])
t = tf.random.normal(shape=[11, 7, 3, 2])
e =  tf.einsum('...ij,...jk->...ik', s, t)
print(e.shape)

(11, 7, 5, 2)
